In [ ]:
import boto3
import pandas as pd
import json
import sagemaker
from sagemaker.inputs import TrainingInput
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel


In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = "creditcard-pipeline-bucket"
prefix = "processed"

train_input = TrainingInput(f"s3://{bucket}/{prefix}/train.csv", content_type="text/csv")
test_input = TrainingInput(f"s3://{bucket}/{prefix}/test.csv", content_type="text/csv")


In [ ]:

sklearn_estimator = SKLearn(
    entry_point="train_model.py",
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.0-1",
    py_version="py3",
    sagemaker_session=sagemaker_session
)

sklearn_estimator.fit({"train": train_input, "test": test_input})


In [ ]:
sk_model = SKLearnModel(
    model_data=sklearn_estimator.model_data,
    role=role,
    entry_point="inference.py",
    framework_version="1.0-1",
    py_version="py3"
)

predictor = sk_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="creditcard-endpoint"
)


In [ ]:
# Example of single transaction
test_data = [
    {"Time": 0, "V1": -1.359807, "V2": -0.072781, "V3": 2.536346, "V4": 1.378155,
     "V5": -0.338321, "V6": 0.462388, "V7": 0.239599, "V8": 0.098698, "V9": 0.363787,
     "V10": 0.090794, "V11": -0.551600, "V12": -0.617801, "V13": -0.991390, "V14": -0.311169,
     "V15": 1.468177, "V16": -0.470401, "V17": 0.207971, "V18": 0.025791, "V19": 0.403993,
     "V20": 0.251412, "V21": -0.018307, "V22": 0.277838, "V23": -0.110474, "V24": 0.066928,
     "V25": 0.128539, "V26": -0.189115, "V27": 0.133558, "V28": -0.021053, "Amount": 149.62}
]

payload = json.dumps(test_data)
runtime = boto3.client('sagemaker-runtime')
endpoint_name = 'creditcard-endpoint'

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=payload
)

result = json.loads(response['Body'].read().decode())
print("Prediction:", result)


In [ ]:
# Load processed test set
test_df = pd.read_csv(f"s3://{bucket}/{prefix}/test.csv")

# Pick a fraud row
fraud_row = test_df[test_df['Class'] == 1].iloc[0]
features_only = fraud_row.drop('Class')

payload = [features_only.values.tolist()]
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)
print("Raw model output:", response['Body'].read().decode())
